In [1]:
import keras
keras.__version__

Using TensorFlow backend.


'2.2.4'

***
# IMDB 데이터셋

### 데이터셋 정보
- 훈련 데이터와 테스트 데이터가 각각 25000개
- 데이터의 각 review에는 label이 붙어 있다.
- 부정은 0, 긍정은 1로 나타낸다.
- 약 50%는 긍정, 50%는 부정 리뷰로 구성
- review 문장의 단어들을 출현빈도순으로 정렬해서 정수로 변환시킨 시퀀스를 x로 한다.
- 스탠포드 대학의 앤드류 마스가 수집했다.

### 예측하고자 하는 방법

- 이진 분류 문제로, 학습 데이터를 이용해, 설계한 신경망을 학습시킨다.
- 크로스엔트로피를 이용해 원본 분포와 예측 분포 사이를 측정한다.
- rmsprop 옵티마이저와 binary_crossentropy 손실 함수로 모델을 설정
- 학습된 신경망을 이용해 어떤 리뷰가 긍정일 확률을 예측할 수 있도록 한다.

### 학습을 위해 데이터가 어떻게 가공/처리 되었는지?
- 25000개의 샘플을 훈련셋 20000개와 5000개의 검증셋으로 분리
- 길이를 맞추기 위해 패딩 삽입
- 임베딩 층을 통과하여 실수 텐서로 매핑
    
### 사용된 모델의 입력층과 출력층

- 입력층
    - 숫자 인덱스를 밀집 벡터로 매핑하는 embedding층
    - (샘플, 시퀀스길이)를 입력 인자로 받음
    - 크기가 (샘플, 시퀀스 길이, 임베딩 차원)인 3D 실수 텐서를 출력함.
- 출력층
    - 은닉 유닛이 1인 dense층

###  사용된 모델의 특징에 대한 기술 
- rmsprop 옵티마이저와 binary_crossentropy 손실 함수로 모델을 컴파일.
- conv1D 층을 사용하여 한 원소의 앞뒤 원소들을 확인해 큰 feature를 뽑아냄
- GlobalMaxPooling이 conv1D층에서 나오는 feature를 받아 가장 큰 벡터하나를 뽑아내줌
***


In [4]:
from keras.datasets import imdb
import numpy as np

max_features = 10000
text_max_words = 500

# save np.load
np_load_old = np.load

# modify the default parameters of np.load
np.load = lambda *a,**k: np_load_old(*a, allow_pickle=True, **k)

# call load_data with allow_pickle implicitly set to true
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)

# restore np.load for future normal usage
np.load = np_load_old
#예제 실행시 오류나서 스택오버플로우사이트 참고했습니다.
#https://stackoverflow.com/questions/55890813/how-to-fix-object-arrays-cannot-be-loaded-when-allow-pickle-false-for-imdb-loa/56062555

In [6]:
from keras.preprocessing import sequence

x_val = x_train[20000:]
y_val = y_train[20000:]
x_train = x_train[:20000]
y_train = y_train[:20000]


x_train = sequence.pad_sequences(x_train, maxlen=text_max_words)
x_val = sequence.pad_sequences(x_val, maxlen=text_max_words)
x_test = sequence.pad_sequences(x_test, maxlen=text_max_words)

#### 신경망 모델 제작

In [12]:
from keras import models
from keras import layers
from keras.models import Sequential
from keras.layers import Embedding, Dropout, Conv1D, GlobalMaxPooling1D, Dense

model = Sequential()
model.add(layers.Embedding(20000, 128, input_length=200))
model.add(layers.Conv1D(32,
                 7,
                 activation='relu'))
model.add(layers.MaxPooling1D())
model.add(layers.Conv1D(32,
                 7,
                 activation='relu'))
model.add(layers.GlobalMaxPooling1D())
model.add(layers.Dense(1))
model.summary()

W0805 20:55:43.598358  5688 deprecation_wrapper.py:119] From C:\Users\a9327\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.



_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 200, 128)          2560000   
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 194, 32)           28704     
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 97, 32)            0         
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 91, 32)            7200      
_________________________________________________________________
global_max_pooling1d_2 (Glob (None, 32)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 33        
Total params: 2,595,937
Trainable params: 2,595,937
Non-trainable params: 0
_________________________________________________________________


In [13]:
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
history = model.fit(x_train,
                    y_train,
                    epochs=25,
                    batch_size=128,
                    validation_data=(x_val, y_val))

model.summary()